In [12]:
import spacy

nlp = spacy.load("en_core_web_sm")

preferencias_usuario = {
    "Resumen": 1,
    "Mapa Conceptual": 2,
    "Ejercicios Prácticos": 3
}

ajuste_metodo = {
    "Resumen": 0,
    "Mapa Conceptual": 1,
    "Ejercicios Prácticos": 2
}

def evaluar_dificultad(bloque):
    longitud = len(bloque.split())
    num_palabras_complejas = sum(1 for word in bloque.split() if len(word) > 8)
    
    dificultad = 1
    if longitud > 20:
        dificultad += 1
    if num_palabras_complejas > 3:
        dificultad += 1
        
    return dificultad

def clasificar_tipo_contenido(bloque):
    if "es" in bloque.lower() or "son" in bloque.lower():
        return "Definición"
    elif "ejemplo" in bloque.lower() or "aplicación" in bloque.lower():
        return "Ejemplo"
    elif "resolver" in bloque.lower() or "calcular" in bloque.lower():
        return "Problema"
    else:
        return "Teoría General"

def asignar_metodo_por_tipo(tipo_contenido, dificultad):
    if tipo_contenido == "Definición" or tipo_contenido == "Teoría General":
        if dificultad <= 2:
            return "Resumen"
        else:
            return "Mapa Conceptual"
    elif tipo_contenido == "Ejemplo":
        return "Mapa Conceptual"
    elif tipo_contenido == "Problema":
        return "Ejercicios Prácticos"
    else:
        return "Resumen"

texto = """
Álgebra Lineal es una rama de las matemáticas que se ocupa del estudio de vectores, espacios vectoriales, 
transformaciones lineales y sistemas de ecuaciones lineales. Un vector es un objeto que tiene tanto magnitud 
como dirección y puede representarse en un espacio n-dimensional. Los espacios vectoriales, también conocidos 
como espacios lineales, son conjuntos de vectores que pueden sumarse entre sí y multiplicarse por escalares para 
producir otro vector en el mismo espacio.
"""

def dividir_texto_en_bloques(texto, num_bloques=3):
    oraciones = [sent.text for sent in nlp(texto).sents]
    bloques = [oraciones[i::num_bloques] for i in range(num_bloques)]
    bloques_texto = [' '.join(bloque) for bloque in bloques]
    return bloques_texto

def procesar_texto(texto, num_bloques=3):
    bloques = dividir_texto_en_bloques(texto, num_bloques)
    
    recomendaciones = {}
    for i, bloque in enumerate(bloques):
        dificultad = evaluar_dificultad(bloque)
        tipo_contenido = clasificar_tipo_contenido(bloque)
        
        metodo = asignar_metodo_por_tipo(tipo_contenido, dificultad)
        score = preferencias_usuario[metodo] + 2 * dificultad + ajuste_metodo[metodo]
        
        recomendaciones[f"Bloque {i+1}"] = {
            "Texto": bloque,
            "Tipo de Contenido": tipo_contenido,
            "Método": metodo,
            "Score": score
        }

    return recomendaciones

recomendaciones = procesar_texto(texto)

for bloque, info in recomendaciones.items():
    print(f"\n{bloque}:")
    print(f"Texto: {info['Texto']}")
    print(f"Tipo de Contenido: {info['Tipo de Contenido']}")
    print(f"Método Sugerido: {info['Método']} (Score: {info['Score']})")



Bloque 1:
Texto: 
Álgebra Lineal es una rama de las matemáticas que se ocupa del estudio de vectores, espacios vectoriales, 
transformaciones lineales y sistemas de ecuaciones lineales. sí y multiplicarse por escalares para 
producir otro vector en el mismo espacio.

Tipo de Contenido: Definición
Método Sugerido: Mapa Conceptual (Score: 9)

Bloque 2:
Texto: Un vector es un objeto que tiene tanto magnitud 
como dirección y puede representarse en un espacio n-dimensional.
Tipo de Contenido: Definición
Método Sugerido: Resumen (Score: 3)

Bloque 3:
Texto: Los espacios vectoriales, también conocidos 
como espacios lineales, son conjuntos de vectores que pueden sumarse entre
Tipo de Contenido: Definición
Método Sugerido: Resumen (Score: 5)
